Note: if you have not run a model (either [logistic regression]("./03-model-logistic-regression.ipynb") or [SVC]("./03-model-svc.ipynb")) notebook, this will not render!

In [122]:
import pandas as pd
import sklearn

pt_info_clean = pd.read_csv("../data/interim/pt_info_clean.csv")

In [123]:
import numpy as np

# change to numpy arrays in order to interact with KSDrift
pt_info_array = np.asarray(pt_info_clean)

from sklearn import model_selection
x_train, x_test, y_train, y_test = model_selection.train_test_split(pt_info_array, \
                                                                    pt_info_array[:,2], #mrsa positive \ 
                                                                    test_size=0.2, \
                                                                    random_state=430)

CEM

In [124]:
from sklearn.linear_model import LogisticRegression

In [125]:
lr = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
lr.fit(x_train, y_train)

from mlworkflows import util
util.serialize_to(lr, "./model.sav")

# loading model
filename = './model.sav'
model = pickle.load(open(filename, 'rb'))

In [126]:
class_names=list(pt_info_clean.mrsa_positive)
idx = 2
X = x_test[idx].reshape((1,) + x_test[idx].shape)
print('Prediction on instance to be explained: {}'.format(class_names[np.argmax(lr.predict(X))]))
print('Prediction probabilities for each class on the instance: {}'.format(lr.predict(X)))

Prediction on instance to be explained: 0.0
Prediction probabilities for each class on the instance: [0.]


In [127]:
x_train.shape

(115, 125)

In [132]:
import alibi
from alibi.explainers import CEM

shape = x_train[1]
mode = 'PN'
cem = CEM(lr.predict(y_train.reshape(-1, 1)), mode, shape)

ValueError: X has 1 features per sample; expecting 125

In [ ]:
cem.fit(x_train, no_info_type='median')  # we need to define what feature values contain the least
                                         # info wrt predictions
                                         # here we will naively assume that the feature-wise median
                                         # contains no info; domain knowledge helps!
explanation = cem.explain(X, verbose=False)

In [ ]:
print('Original instance: {}'.format(explanation.X))
print('Predicted class: {}'.format(class_names[explanation.X_pred]))

In [ ]:
print('Pertinent negative: {}'.format(explanation.PN))
print('Predicted class: {}'.format(class_names[explanation.PN_pred]))